# Tutorial goal

This tutorial aims to show how to **configure** a **test environment** using
the **TestEnv module** provided by LISA.

# Configure logging

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [2]:
# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [3]:
# Other python modules required by this notebook
import json
import time
import os

# Test environment setup

## Do you have custom scripts to deploy and use on target?

In [4]:
# Custom scrips must be deployed under $LISA_HOME/tools
!tree ../../tools

../../tools
├── arm64
│   ├── perf
│   ├── rt-app
│   ├── taskset
│   └── trace-cmd
├── armeabi
│   ├── busybox
│   ├── daemonize
│   ├── htop
│   ├── htop.armv7
│   ├── perf
│   ├── rt-app
│   ├── sysbench
│   ├── taskset
│   ├── terminfo.tar.bz2
│   └── trace-cmd
├── LICENSE.busybox
├── LICENSE.perf
├── LICENSE.rt-app
├── LICENSE.taskset
├── LICENSE.trace-cmd
├── plots.py
├── report.py
├── report.pyc
├── scripts
│   ├── cgroup_run_into.sh
│   ├── cgroup_tasks_move.sh
│   ├── cpuidle_sampling.sh
│   ├── odroid_sampler.py
│   ├── odroid_sampler.sh
│   ├── trace_frequencies.sh
│   └── traceview_extract.awk
├── x86
│   ├── busybox
│   └── chrt
└── x86_64
    ├── busybox
    ├── chrt
    ├── perf
    ├── rt-app
    ├── taskset
    ├── trace-cmd
    └── wlg

5 directories, 38 files


In [5]:
# This is the (not so fancy) script we want to deploy
!cat ../../tools/scripts/cpuidle_sampling.sh

#!/bin/bash

# CPU to monitor
CPU=${1:-0}
# Sampling time
SLEEP=${2:-1}
# Samples to collect
COUNT=${3:-3}

# Enter CPU's sysfs
cd /sys/devices/system/cpu

# Initial C-State residencies counter
ISC=$(find cpu0/cpuidle -name "state*" | wc -l)
for I in $(seq 0 $((ISC-1))); do
	LCS[$I]=`cat cpu$CPU/cpuidle/state$I/usage`
done

# Dump header
printf "#%13s " "Time"
for I in $(seq 0 $((ISC-1))); do
  printf "%14s " "idle$I"
done
echo

# Sampling loop
for I in $(seq $COUNT); do

	sleep $SLEEP

	# Dump CPU C-State residencies
	now=$(date +%s)
	printf "%14d " $now
	for I in $(seq 0 $((ISC-1))); do
		U=`cat cpu$CPU/cpuidle/state$I/usage`
		CCS=$(($U - ${LCS[$I]}))
		printf "%14d " $CCS
		LCS[$I]=$U
	done
	echo


done

# vim: ts=2


## Which devlib modules you need for your experiments?

In [6]:
# You can have a look at the devlib supported modules by lising the
devlib_modules_folder = 'libs/devlib/devlib/module/'

logging.info("Devlib provided modules are found under:")
logging.info("   $LISA_HOME/{}".format(devlib_modules_folder))
!cd ../../ ; find {devlib_modules_folder} -name "*.py" | sed 's|libs/devlib/devlib/module/|   |' | grep -v __init__ 

04:34:06  INFO    : Devlib provided modules are found under:
04:34:06  INFO    :    $LISA_HOME/libs/devlib/devlib/module/


   thermal.py
   vexpress.py
   cooling.py
   android.py
   biglittle.py
   cpuidle.py
   hotplug.py
   cpufreq.py
   hwmon.py
   cgroups.py


## Setup you TestEnv confguration

In [7]:
# Setup a target configuration
conf = {

    # Define the kind of target platform to use for the experiments
    "platform"    : 'linux',  # platform type, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno',   # board type, valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board

    # Login credentials
    "host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",

    # Custom tools to deploy on target, they must be placed under:
    #   $LISA_HOME/tools/(ARCH|scripts)
    "tools" : [ "cpuidle_sampling.sh" ],

    # FTrace configuration
    "ftrace" : {
         "events" : [
             "cpu_idle",
             "sched_switch",
         ],
         "buffsize" : 10240,
    },
    
    # Where results are collected
    "results_dir" : "TestEnvExample",
    
    # Devlib module required (or not required)
    'modules' : [ "cpufreq", "cgroups" ],
    #"exclude_modules" : [ "hwmon" ],
    
    # Local installation path used for kernel/dtb installation on target
    # The specified path MUST be accessible from the board, e.g.
    # - JUNO/TC2: it can be the mount path of the VMESD disk image
    # - Other board: it can be a TFTP server path used by the board bootloader
    "tftp"  : {
        "folder"    : "/var/lib/tftpboot",
        "kernel"    : "kern.bin",
        "dtb"       : "dtb.bin",
    },

}

In [8]:
from env import TestEnv

# Initialize a test environment using the provided configuration
te = TestEnv(conf)

04:34:08  INFO    :         Target - Using base path: /home/derkling/Code/lisa
04:34:08  INFO    :         Target - Loading custom (inline) target configuration
04:34:08  DEBUG   :         Target - Target configuration {'username': 'root', 'platform': 'linux', 'host': '192.168.0.1', 'ftrace': {'buffsize': 10240, 'events': ['cpu_idle', 'sched_switch']}, 'board': 'juno', 'modules': ['cpufreq', 'cgroups'], 'tftp': {'kernel': 'kern.bin', 'folder': '/var/lib/tftpboot', 'dtb': 'dtb.bin'}, 'password': '', 'tools': ['cpuidle_sampling.sh'], 'results_dir': 'TestEnvExample'}
04:34:08  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'cgroups', 'hwmon']
04:34:08  INFO    :         Target - Connecting linux target:
04:34:08  INFO    :         Target -   username : root
04:34:08  INFO    :         Target -       host : 192.168.0.1
04:34:08  INFO    :         Target -   password : 
04:34:08  DEBUG   :         Target - Setup LINUX target...
04:34:08  DEBUG   : Installing module vex

## Attributes

The initialization of the test environment pre-initialize some useful<br>
environment variables which are available to write test cases.

These are some of the information available via the TestEnv object.

In [9]:
# The complete configuration of the target we have configured
print json.dumps(te.conf, indent=4)

{
    "username": "root", 
    "ftrace": {
        "buffsize": 10240, 
        "events": [
            "cpu_idle", 
            "sched_switch"
        ]
    }, 
    "host": "192.168.0.1", 
    "password": "", 
    "tools": [
        "cpuidle_sampling.sh", 
        "trace-cmd"
    ], 
    "modules": [
        "cpufreq", 
        "cgroups"
    ], 
    "results_dir": "TestEnvExample", 
    "platform": "linux", 
    "board": "juno", 
    "__features__": [], 
    "tftp": {
        "kernel": "kern.bin", 
        "folder": "/var/lib/tftpboot", 
        "dtb": "dtb.bin"
    }
}


In [10]:
# Last configured kernel and DTB image
print te.kernel
print te.dtb

None
None


In [11]:
# The IP and MAC address of the target
print te.ip
print te.mac

192.168.0.1
None


In [12]:
# A full platform descriptor
print json.dumps(te.platform, indent=4)

{
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 64
            }, 
            "cpu": {
                "cap_max": 1024, 
                "nrg_max": 616
            }
        }, 
        "little": {
            "cluster": {
                "nrg_max": 57
            }, 
            "cpu": {
                "cap_max": 447, 
                "nrg_max": 93
            }
        }
    }, 
    "clusters": {
        "big": [
            1, 
            2
        ], 
        "little": [
            0, 
            3, 
            4, 
            5
        ]
    }, 
    "cpus_count": 6, 
    "freqs": {
        "big": [
            450000, 
            625000, 
            800000, 
            950000, 
            1100000
        ], 
        "little": [
            450000, 
            575000, 
            700000, 
            775000, 
            850000
        ]
    }, 
    "topology": [
        [
            0, 
            3, 
            4, 
        

In [13]:
# A pre-created folder to host the tests results generated using this
# test environment, notice that the suite could add additional information
# in this folder, like for example a copy of the target configuration
# and other target specific collected information
te.res_dir

'/home/derkling/Code/lisa/results/TestEnvExample'

In [14]:
# The working directory on the target
te.workdir

'/data/local/schedtest'

## Functions

Some methods are also exposed to test developers which could be used to easy
the creation of tests.

These are some of the methods available:

In [15]:
# Calibrate RT-App (if required) and get the most updated calibration value
te.calibration()

04:27:32  DEBUG   : No RT-App workloads, skipping calibration


In [16]:
# Generate a JSON file with the complete platform description
te.platform_dump(dest_dir='/tmp')

04:27:32  DEBUG   :       Platform - Dump platform descriptor in [/tmp/platform.json]


({'clusters': {'big': [1, 2], 'little': [0, 3, 4, 5]},
  'cpus_count': 6,
  'freqs': {'big': [450000, 625000, 800000, 950000, 1100000],
   'little': [450000, 575000, 700000, 775000, 850000]},
  'nrg_model': {u'big': {u'cluster': {u'nrg_max': 64},
    u'cpu': {u'cap_max': 1024, u'nrg_max': 616}},
   u'little': {u'cluster': {u'nrg_max': 57},
    u'cpu': {u'cap_max': 447, u'nrg_max': 93}}},
  'topology': [[0, 3, 4, 5], [1, 2]]},
 '/tmp/platform.json')

In [17]:
# Force a reboot of the target (and wait specified [s] before reconnect)
# te.reboot(reboot_time=60, ping_time=15)

In [18]:
# Resolve a MAC address into an IP address
# te.resolv_host(host='00:02:F7:00:5A:5B')

In [19]:
# Copy the specified file into the TFTP server folder defined by configuration
te.tftp_deploy('/etc/group')

04:27:33  INFO    :           TFTP - Deploy /etc/group into /var/lib/tftpboot/group


In [20]:
!ls -la /var/lib/tftpboot

total 12
drwxrwxrwx  2 root     nogroup  4096 Feb 22 17:34 .
drwxr-xr-x 79 root     root     4096 Feb 26 15:17 ..
-rw-r--r--  1 derkling derkling 1168 Mar  2 16:27 group


# Access to the devlib API

A special TestEnv attribute is <b>target</b>, which represents a <b>devlib instance</b>.
Using the target attribute we can access to the full set of devlib provided
functionalities. Which are summarized in the following sections.

## Remotes commands execution

In [22]:
# Run a command on the target
te.target.execute("echo -n 'Hello Test Environment'", as_root=False)

04:27:34  DEBUG   : echo -n 'Hello Test Environment'


'Hello Test Environment'

In [23]:
# Spawn a command in background on the target
logging.info("Spawn a task which will run for a while...")
process = te.target.kick_off("sleep 10", as_root=True)

04:27:34  INFO    : Spawn a task which will run for a while...
04:27:35  DEBUG   : sudo -- sh -c 'sh -c "sleep 10" 1>/dev/null 2>/dev/null &'


In [24]:
output = te.target.execute("ps")
print '\n'.join(output.splitlines())

04:27:35  DEBUG   : ps


  PID TTY          TIME CMD
 6097 pts/0    00:00:00 sh
 6270 pts/0    00:00:00 sleep
 6271 pts/0    00:00:00 ps


Notice that _the Shell PID is always the same_ for all commands we execute.<br>
This is due to devlib ensuring to keep a **persistent connection** with the target device.

## Running custom scripts

In [27]:
my_script = te.target.get_installed("cpuidle_sampling.sh")
print my_script

04:28:18  DEBUG   : if [ -e '/root/devlib-target/bin' ]; then echo 1; else echo 0; fi
04:28:18  DEBUG   : ls -1 /root/devlib-target/bin


/root/devlib-target/bin/cpuidle_sampling.sh


In [28]:
output = te.target.execute(my_script, as_root=True)
output.splitlines()

04:28:19  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/cpuidle_sampling.sh'


['#         Time          idle0          idle1          idle2 ',
 '    1456934283              0             11             19 ',
 '    1456934284              0              5             17 ',
 '    1456934285              0              5             19 ']

Notice that the output is returned as a **list of lines**. This provides a useful base for post-processing the output of that command.

In [35]:
# We can also use "notebook embedded" scripts
# my_script = " \
# for I in $(seq 3); do \
#     grep '' /sys/devices/system/cpu/cpu*/cpufreq/stats/time_in_stats | \
#     sed -e 's|/sys/devices/system/cpu/cpu||' -e 's|/cpufreq/scaling_governor:| |' \
#     sleep 1 \
# done \
# "

In [34]:
# print te.target.execute(my_script)

## Access to target specific attributes

In [9]:
# Acces to many target specific information
print "ABI                 : ", te.target.abi
print "big Core Family     : ", te.target.big_core
print "LITTLE Core Family  : ", te.target.little_core
print "CPU's Clusters IDs  : ", te.target.core_clusters
print "CPUs type           : ", te.target.core_names

ABI                 :  arm64
big Core Family     :  A57
LITTLE Core Family  :  A53
CPU's Clusters IDs  :  [0, 1, 1, 0, 0, 0]
CPUs type           :  ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']


In [10]:
# Access to big.LITTLE specific information
print "big CPUs IDs        : ", te.target.bl.bigs
print "LITTLE CPUs IDs     : ", te.target.bl.littles
print "big CPUs freqs      : {}".format(te.target.bl.get_bigs_frequency())
print "big CPUs governor   : {}".format(te.target.bl.get_bigs_governor())

04:35:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
04:35:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'\'''


big CPUs IDs        :  [1, 2]
LITTLE CPUs IDs     :  [0, 3, 4, 5]
big CPUs freqs      : 1100000

04:35:04  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
04:35:04  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_governor'\'''



big CPUs governor   : performance


## Modules usage example: CPUFreq

In [11]:
# You can use autocompletion to have a look at the supported method for a
# specific module
te.target.cpufreq #.get_all_governors()

04:36:33  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_get_all_governors'


{'0': 'performance',
 '1': 'performance',
 '2': 'performance',
 '3': 'performance',
 '4': 'performance',
 '5': 'performance'}

In [16]:
# Get goverors available for CPU0
te.target.cpufreq.list_governors(0)

04:39:00  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_available_governors'\'''


['conservative', 'ondemand', 'userspace', 'powersave', 'performance', 'sched']

In [17]:
# Set the "ondemand" governor
te.target.cpufreq.set_governor(0, 'ondemand')

04:40:04  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_available_governors'\'''
04:40:04  DEBUG   : sudo -- sh -c 'echo ondemand > '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:40:05  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:40:05  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:40:06  DEBUG   : ls -1 /sys/devices/system/cpu/cpu0/cpufreq/ondemand


In [19]:
# Check governor tunables
te.target.cpufreq.get_governor_tunables(0)

04:40:29  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:40:29  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:40:30  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/ignore_nice_load'\'''
04:40:30  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/io_is_busy'\'''
04:40:31  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/powersave_bias'\'''
04:40:31  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/sampling_down_factor'\'''
04:40:31  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/sampling_rate'\'''
04:40:32  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/sampling_rate_min'\'''
04:40:32  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/up_threshold'\'''


{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '1200000',
 'sampling_rate_min': '24000',
 'up_threshold': '95'}

In [22]:
# Update governor tunables
te.target.cpufreq.set_governor_tunables(0, sampling_rate=2000000)
te.target.cpufreq.get_governor_tunables(0)

04:41:46  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:41:46  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:41:47  DEBUG   : sudo -- sh -c 'echo 2000000 > '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/sampling_rate'\'''
04:41:47  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/sampling_rate'\'''
04:41:47  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:41:48  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\'''
04:41:48  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/ignore_nice_load'\'''
04:41:49  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/io_is_busy'\'''
04:41:49  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu0/cpufreq/ondemand/powersave_bias'\'''
04:41:49  DEBUG   : sudo -- sh -c 'cat '

{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '2000000',
 'sampling_rate_min': '24000',
 'up_threshold': '95'}

## Modules usage example: CGroups

In [44]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    print '{:10} (hierarchy id: {:d}) has {} cgroups'.format(n, h, g)

04:51:18  INFO    :         CGroup - Available controllers:
04:51:18  DEBUG   : /root/devlib-target/bin/busybox cat /proc/cgroups


cpuset     (hierarchy id: 1) has 2 cgroups
cpu        (hierarchy id: 2) has 1 cgroups
schedtune  (hierarchy id: 3) has 1 cgroups
memory     (hierarchy id: 4) has 1 cgroups
devices    (hierarchy id: 5) has 1 cgroups
freezer    (hierarchy id: 6) has 1 cgroups
perf_event (hierarchy id: 7) has 1 cgroups
hugetlb    (hierarchy id: 8) has 1 cgroups
pids       (hierarchy id: 9) has 1 cgroups


In [23]:
# Get a reference to the CPUSet controller
cpuset = target.cgroups.controller('cpuset')

In [47]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
print 'Existing CGropups:'
for cg in cgroups:
    print "   ", cg

04:51:42  DEBUG   : Listing groups for cpuset controller
04:51:43  DEBUG   : /root/devlib-target/bin/busybox find /sys/fs/cgroup/devlib_cpuset -type d
04:51:43  DEBUG   : Populate cpuset cgroup: /
04:51:43  DEBUG   : Populate cpuset cgroup: /LITTLE


Existing CGropups:
    /
    /LITTLE


In [29]:
# Create a LITTLE partition and check which tunables we have
cpuset_littles = cpuset.cgroup('/LITTLE')
cpuset_littles.get()

04:45:15  DEBUG   : Reading cpuset attributes from:
04:45:15  DEBUG   :   /sys/fs/cgroup/devlib_cpuset/LITTLE
04:45:15  DEBUG   : /root/devlib-target/bin/shutils cgroups_get_attributes /sys/fs/cgroup/devlib_cpuset/LITTLE cpuset


{'cpu_exclusive': '0',
 'cpus': '',
 'effective_cpus': '',
 'effective_mems': '',
 'mem_exclusive': '0',
 'mem_hardwall': '0',
 'memory_migrate': '0',
 'memory_pressure': '0',
 'memory_spread_page': '0',
 'memory_spread_slab': '0',
 'mems': '',
 'sched_load_balance': '1',
 'sched_relax_domain_level': '-1'}

In [33]:
# Setup CPUs and MEMORY nodes for the LITTLE partition
cpuset_littles.set(cpus=te.target.bl.littles, mems=0)

04:47:33  DEBUG   : Set attribute [/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.cpus] to: 0,3-5"
04:47:33  DEBUG   : sudo -- sh -c 'echo 0,3-5 > '\''/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.cpus'\'''
04:47:33  DEBUG   : sudo -- sh -c 'cat '\''/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.cpus'\'''
04:47:34  DEBUG   : Set attribute [/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.mems] to: 0"
04:47:34  DEBUG   : sudo -- sh -c 'echo 0 > '\''/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.mems'\'''
04:47:34  DEBUG   : sudo -- sh -c 'cat '\''/sys/fs/cgroup/devlib_cpuset/LITTLE/cpuset.mems'\'''


In [52]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    cpus = attrs['cpus']
    print '{}:{:<15} cpus: {}'.format(cpuset.kind, cgroup.name, cpus)

04:52:55  DEBUG   : Reading cpuset attributes from:
04:52:55  DEBUG   :   /sys/fs/cgroup/devlib_cpuset
04:52:55  DEBUG   : /root/devlib-target/bin/shutils cgroups_get_attributes /sys/fs/cgroup/devlib_cpuset cpuset
04:52:55  DEBUG   : Reading cpuset attributes from:
04:52:55  DEBUG   :   /sys/fs/cgroup/devlib_cpuset/LITTLE
04:52:55  DEBUG   : /root/devlib-target/bin/shutils cgroups_get_attributes /sys/fs/cgroup/devlib_cpuset/LITTLE cpuset


cpuset:/               cpus: 0-5
cpuset:/LITTLE         cpus: 0,3-5


In [53]:
# Methods exists to move tasks in/out and in between groups
# cpuset_littles.add_task()

# Sample energy from the target

In [6]:
# Reset and sample energy counters
te.emeter.reset()

# Sleep some time
time.sleep(2)

# Sample energy consumption since last reset
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "First read: ", nrg

# Sleep some more time
time.sleep(2)

# Sample again
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "Second read: ", nrg

03:02:39  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
03:02:39  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
03:02:39  DEBUG   : SAMPLE: {'a53': {'total': 10.111351999999897, 'last': 1335.646094, 'delta': 9.059672999999975}, 'a57': {'total': 15.980763000000024, 'last': 2052.693486, 'delta': 13.449973000000227}}
03:02:39  DEBUG   : RESET: {'a53': {'total': 0, 'last': 1335.646094, 'delta': 0}, 'a57': {'total': 0, 'last': 2052.693486, 'delta': 0}}
03:02:40  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
03:02:40  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
03:02:40  DEBUG   : SAMPLE: {'a53': {'total': 0.3706250000000182, 'last': 1336.016719, 'delta': 0.3706250000000182}, 'a57': {'total': 0.49970399999983783, 'last': 2053.19319, 'delta': 0.49970399999983783}}
03:02:40  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
03:02:41  DEBUG   :

First read:  {
    "a53": {
        "total": 0.7240460000000439, 
        "last": 1336.37014, 
        "delta": 0.35342100000002574
    }, 
    "a57": {
        "total": 0.9984340000000884, 
        "last": 2053.69192, 
        "delta": 0.4987300000002506
    }
}
Second read:  {
    "a53": {
        "total": 1.4408379999999852, 
        "last": 1337.086932, 
        "delta": 0.19358299999998962
    }, 
    "a57": {
        "total": 2.4585919999999533, 
        "last": 2055.152078, 
        "delta": 0.4880590000002485
    }
}


# Configure FTrace for a sepcific experiment

In [7]:
# Configure a specific set of events to trace
te.ftrace_conf(
    {                                                                                                                                             
         "events" : [                                                                                                                                            
             "cpu_idle",                                                                                                                                         
             "cpu_capacity",
             "cpu_frequency",
             "sched_switch",
         ],                                                                                                                                                      
         "buffsize" : 10240                                                                                                                                      
    }
)

02:59:43  DEBUG   : /usr/bin/scp -r   /home/derkling/Code/lisa/libs/devlib/devlib/bin/arm64/trace-cmd root@192.168.0.1:/root/devlib-target/bin/trace-cmd
02:59:44  DEBUG   : chmod a+x /root/devlib-target/bin/trace-cmd
02:59:44  DEBUG   : cat /sys/kernel/debug/tracing/available_events
02:59:44  INFO    :         FTrace - Enabled tracepoints:
02:59:44  INFO    :         FTrace -   cpu_idle
02:59:44  INFO    :         FTrace -   cpu_capacity
02:59:44  INFO    :         FTrace -   cpu_frequency
02:59:44  INFO    :         FTrace -   sched_switch


In [8]:
# Start/Stop a FTrace session
te.ftrace.start()
te.target.execute("uname -a")
te.ftrace.stop()

02:59:45  DEBUG   : sudo -- sh -c 'echo 10240 > '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
02:59:45  DEBUG   : sudo -- sh -c 'cat '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
02:59:45  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd reset'
02:59:46  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd start -e cpu_idle -e cpu_capacity -e cpu_frequency -e sched_switch'
02:59:48  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_START > '\''/sys/kernel/debug/tracing/trace_marker'\'''
02:59:48  DEBUG   : Trace CPUFreq frequencies
02:59:48  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
02:59:48  DEBUG   : uname -a
02:59:49  DEBUG   : Trace CPUFreq frequencies
02:59:49  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
02:59:49  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_STOP > '\''/sys/kernel/debug/tracing/trace_marker'\'''
02:59:49  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-c

In [9]:
# Collect and visualize the trace
trace_file = os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
output = os.popen("DISPLAY=:0.0 kernelshark {}".format(trace_file))

02:59:50  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd extract -o /root/devlib-target/trace.dat'
02:59:51  DEBUG   : /usr/bin/scp -r   root@192.168.0.1:/root/devlib-target/trace.dat /home/derkling/Code/lisa/results/TestEnvExample/trace.dat
